In [ ]:
# imports and libs
!pip3 -q install transformers
!pip3 install trl
!pip3 install -U bitsandbytes
!pip3 install datasets
!pip3 install peft
!pip3 install accelerate
# !pip install -q datasets trl peft bitsandbytes accelerate datasets
# !pip3 install transformers==4.46.2 peft==0.13.2 accelerate==1.1.1 trl==0.12.1 bitsandbytes==0.45.2 datasets==3.1.0 huggingface-hub==0.26.2 safetensors==0.4.5 pandas==2.2.2 matplotlib==3.8.0 numpy==1.26.4

import torch
from transformers import AutoModelForCausalLM, DataCollatorWithPadding
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig
from torch.optim import AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, TaskType
from transformers import Trainer, TrainingArguments
from trl import SFTConfig, SFTTrainer
from peft import PeftModel, PeftConfig

Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.46.2-py3-none-any.whl.metadata (44 kB)
  Using cached peft-0.13.2-py3-none-any.whl.metadata (13 kB)
  Using cached accelerate-1.1.1-py3-none-any.whl.metadata (19 kB)
  Using cached trl-0.12.1-py3-none-any.whl.metadata (10 kB)
ERROR: Could not find a version that satisfies the requirement bitsandbytes==0.45.2 (from versions: 0.31.8, 0.32.0, 0.32.1, 0.32.2, 0.32.3, 0.33.0, 0.33.1, 0.34.0, 0.35.0, 0.35.1, 0.35.2, 0.35.3, 0.35.4, 0.36.0, 0.36.0.post1, 0.36.0.post2, 0.37.0, 0.37.1, 0.37.2, 0.38.0, 0.38.0.post1, 0.38.0.post2, 0.38.1, 0.39.0, 0.39.1, 0.40.0, 0.40.0.post1, 0.40.0.post2, 0.40.0.post3, 0.40.0.post4, 0.40.1, 0.40.1.post1, 0.40.2, 0.41.0, 0.41.1, 0.41.2, 0.41.2.post1, 0.41.2.post2, 0.41.3, 0.41.3.post1, 0.41.3.post2, 0.42.0)
ERROR: No matching distribution found for bitsandbytes==0.45.2


In [3]:
peft_model_id = "./ai_model/"

config = PeftConfig.from_pretrained(peft_model_id)

bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.float32
)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    device_map='auto',
    quantization_config=bnb_config,
    offload_folder="offload", offload_state_dict = True
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [12]:
def generate(model, tokenizer, prompt, max_new_tokens=64, skip_special_tokens=False):
    tokenized_input = tokenizer(prompt, add_special_tokens=False, return_tensors="pt").to(model.device)

    model.eval()
    generation_output = model.generate(**tokenized_input,
                                       eos_token_id=tokenizer.eos_token_id,
                                       max_new_tokens=max_new_tokens)

    output = tokenizer.batch_decode(generation_output,
                                    skip_special_tokens=skip_special_tokens)
    # output
    output = output[0].split("<|assistant|>")[1]
    return output

In [13]:
def gen_prompt(tokenizer, sentence):
    converted_sample = [
        {"role": "user", "content": sentence},
    ]
    prompt = tokenizer.apply_chat_template(converted_sample,
                                           tokenize=False,
                                           add_generation_prompt=True)
    return prompt

In [14]:
user_input = ""
while user_input is not "q" or user_input is not "quit":
  print("Enter q/quit to stop:")
  user_input = input("Enter something to ask doctor:")
  if user_input == "q" or user_input == "quit":
    print("Thank you, goodbye.")
    break
  print(generate(model, tokenizer, gen_prompt(tokenizer, user_input)))

<>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-14-386a3b04eadd>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  while user_input is not "q" or user_input is not "quit":
<ipython-input-14-386a3b04eadd>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  while user_input is not "q" or user_input is not "quit":
<ipython-input-14-386a3b04eadd>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  while user_input is not "q" or user_input is not "quit":
<ipython-input-14-386

Enter q/quit to stop:
Enter something to ask doctor:hey
 Hello, Welcome to Chat Doctor, I don't know if for sure this is a boil or a swollen lymph node in response to a wound or infection that you have on the lower limbs. If you have no wound, then this is most likely a swollen
Enter q/quit to stop:


KeyboardInterrupt: Interrupted by user